# PREDICT THE BURNED AREA OF FOREST FIRES WITH NEURAL NETWORKS

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

In [4]:
# load the dataset
data = pd.read_csv('forestfires.csv')
data.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small


# EDA

In [7]:
data1 = data.iloc[:,2:]


In [8]:
data1.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,...,0,0,0,0,1,0,0,0,0,small
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,small
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,small
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,...,0,0,0,0,1,0,0,0,0,small
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,small


In [9]:
data1['size_category']=data1['size_category'].astype('category')
data1['size_category']=data1['size_category'].cat.codes

In [10]:
data1

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,...,0,0,0,0,1,0,0,0,0,1
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,...,0,0,0,0,0,0,0,1,0,1
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,...,0,0,0,0,0,0,0,1,0,1
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,...,0,0,0,0,1,0,0,0,0,1
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,...,0,0,0,0,0,0,0,0,0,0
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,...,0,0,0,0,0,0,0,0,0,0
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,...,0,0,0,0,0,0,0,0,0,0
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
data1.shape

(517, 29)

In [13]:
data1.describe()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,...,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292,0.164410,...,0.038685,0.003868,0.061896,0.032882,0.104449,0.003868,0.001934,0.029014,0.332689,0.731141
std,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818,0.371006,...,0.193029,0.062137,0.241199,0.178500,0.306138,0.062137,0.043980,0.168007,0.471632,0.443796
min,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FFMC           517 non-null    float64
 1   DMC            517 non-null    float64
 2   DC             517 non-null    float64
 3   ISI            517 non-null    float64
 4   temp           517 non-null    float64
 5   RH             517 non-null    int64  
 6   wind           517 non-null    float64
 7   rain           517 non-null    float64
 8   area           517 non-null    float64
 9   dayfri         517 non-null    int64  
 10  daymon         517 non-null    int64  
 11  daysat         517 non-null    int64  
 12  daysun         517 non-null    int64  
 13  daythu         517 non-null    int64  
 14  daytue         517 non-null    int64  
 15  daywed         517 non-null    int64  
 16  monthapr       517 non-null    int64  
 17  monthaug       517 non-null    int64  
 18  monthdec  

In [15]:
#correlation matrices
data1.corr()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
FFMC,1.000000,0.382619,0.330512,0.531805,0.431532,-0.300995,-0.028485,0.056702,0.040122,0.019306,...,-0.281535,-0.454771,0.031833,-0.040634,-0.074327,-0.037230,-0.088964,-0.005998,0.076609,-0.022063
DMC,0.382619,1.000000,0.682192,0.305128,0.469594,0.073795,-0.105342,0.074790,0.072994,-0.012010,...,-0.317899,-0.105647,-0.001946,-0.050403,-0.407404,-0.081980,-0.074218,-0.187632,0.110907,-0.034715
DC,0.330512,0.682192,1.000000,0.229154,0.496208,-0.039192,-0.203466,0.035861,0.049383,-0.004220,...,-0.399277,-0.115064,-0.100887,-0.186183,-0.650427,-0.114209,-0.078380,0.093279,0.531857,-0.019428
ISI,0.531805,0.305128,0.229154,1.000000,0.394287,-0.132517,0.106826,0.067668,0.008258,0.046695,...,-0.249777,-0.103588,0.020982,0.111516,-0.143520,-0.060493,-0.076559,-0.071154,-0.068877,0.008726
temp,0.431532,0.469594,0.496208,0.394287,1.000000,-0.527390,-0.227116,0.069491,0.097844,-0.071949,...,-0.320015,-0.146520,0.142588,0.051015,-0.341797,-0.045540,-0.053798,-0.053513,0.088006,-0.006021
RH,-0.300995,0.073795,-0.039192,-0.132517,-0.527390,1.000000,0.069410,0.099751,-0.075519,0.064506,...,0.140430,0.170923,0.013185,0.009382,-0.089836,0.086822,-0.035885,-0.072334,-0.062596,0.045243
wind,-0.028485,-0.105342,-0.203466,0.106826,-0.227116,0.069410,1.000000,0.061119,0.012317,0.118090,...,-0.029431,-0.070245,-0.040645,0.012124,0.181433,0.015054,0.011864,-0.053850,-0.181476,-0.059113
rain,0.056702,0.074790,0.035861,0.067668,0.069491,0.099751,0.061119,1.000000,-0.007366,-0.004261,...,-0.014698,-0.004566,-0.013390,-0.013510,-0.020744,-0.004566,-0.003225,-0.012665,-0.051733,-0.050001
area,0.040122,0.072994,0.049383,0.008258,0.097844,-0.075519,0.012317,-0.007366,1.000000,-0.052911,...,-0.020732,-0.012589,0.006149,-0.020314,-0.045596,0.006264,-0.008893,-0.016878,0.056573,-0.311322
dayfri,0.019306,-0.012010,-0.004220,0.046695,-0.071949,0.064506,0.118090,-0.004261,-0.052911,1.000000,...,0.046323,-0.027643,-0.048969,0.006000,0.036205,0.056423,-0.019527,-0.045585,0.107671,0.021810


In [16]:
# data converting(array format)
data2=data1.values

In [17]:
# spilt into input x and y output
x= data2[:,0:28]
y=data2[:,28]

In [18]:
x

array([[ 86.2,  26.2,  94.3, ...,   0. ,   0. ,   0. ],
       [ 90.6,  35.4, 669.1, ...,   0. ,   1. ,   0. ],
       [ 90.6,  43.7, 686.9, ...,   0. ,   1. ,   0. ],
       ...,
       [ 81.6,  56.7, 665.6, ...,   0. ,   0. ,   0. ],
       [ 94.4, 146. , 614.7, ...,   0. ,   0. ,   0. ],
       [ 79.5,   3. , 106.7, ...,   1. ,   0. ,   0. ]])

In [19]:
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# Artificial Nural Network

In [20]:
# create model 
model = Sequential()
model.add(Dense(12, input_dim =28,kernel_initializer='he_uniform',activation='relu'))
model.add(Dense(8, kernel_initializer ='he_uniform',activation ='relu'))
model.add(Dense(1,kernel_initializer ='he_uniform',activation='sigmoid'))

In [61]:
# compile model
model.compile(loss =  'binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [62]:
# fit the model
model.fit(x,y, validation_split = 0.33, epochs = 150, batch_size = 20)

Epoch 1/150
18/18 [==============================] - 1s 16ms/step - loss: 143.4765 - accuracy: 0.2543 - val_loss: 111.0384 - val_accuracy: 0.3626
Epoch 2/150
18/18 [==============================] - 0s 4ms/step - loss: 101.1518 - accuracy: 0.3324 - val_loss: 69.8499 - val_accuracy: 0.3743
Epoch 3/150
18/18 [==============================] - 0s 7ms/step - loss: 59.2261 - accuracy: 0.3410 - val_loss: 30.3217 - val_accuracy: 0.3626
Epoch 4/150
18/18 [==============================] - 0s 5ms/step - loss: 18.9645 - accuracy: 0.3584 - val_loss: 6.9539 - val_accuracy: 0.5848
Epoch 5/150
18/18 [==============================] - 0s 6ms/step - loss: 5.9659 - accuracy: 0.7197 - val_loss: 7.9247 - val_accuracy: 0.6784
Epoch 6/150
18/18 [==============================] - 0s 5ms/step - loss: 4.6040 - accuracy: 0.6908 - val_loss: 5.6377 - val_accuracy: 0.4620
Epoch 7/150
18/18 [==============================] - 0s 5ms/step - loss: 2.5390 - accuracy: 0.6040 - val_loss: 7.3889 - val_accuracy: 0.5146
Ep

Epoch 59/150
18/18 [==============================] - 0s 5ms/step - loss: 0.1059 - accuracy: 0.9595 - val_loss: 1.3782 - val_accuracy: 0.7251
Epoch 60/150
18/18 [==============================] - 0s 4ms/step - loss: 0.0886 - accuracy: 0.9798 - val_loss: 1.3320 - val_accuracy: 0.7310
Epoch 61/150
18/18 [==============================] - 0s 4ms/step - loss: 0.0819 - accuracy: 0.9711 - val_loss: 1.1960 - val_accuracy: 0.7310
Epoch 62/150
18/18 [==============================] - 0s 4ms/step - loss: 0.0849 - accuracy: 0.9682 - val_loss: 1.1241 - val_accuracy: 0.7310
Epoch 63/150
18/18 [==============================] - 0s 4ms/step - loss: 0.0977 - accuracy: 0.9740 - val_loss: 1.2161 - val_accuracy: 0.7310
Epoch 64/150
18/18 [==============================] - 0s 4ms/step - loss: 0.0789 - accuracy: 0.9740 - val_loss: 1.1383 - val_accuracy: 0.7368
Epoch 65/150
18/18 [==============================] - 0s 4ms/step - loss: 0.0796 - accuracy: 0.9740 - val_loss: 1.0576 - val_accuracy: 0.7427
Epoch 

18/18 [==============================] - 0s 4ms/step - loss: 0.0464 - accuracy: 0.9942 - val_loss: 0.6574 - val_accuracy: 0.7953
Epoch 117/150
18/18 [==============================] - 0s 4ms/step - loss: 0.0457 - accuracy: 0.9855 - val_loss: 0.5978 - val_accuracy: 0.8187
Epoch 118/150
18/18 [==============================] - 0s 4ms/step - loss: 0.0443 - accuracy: 0.9884 - val_loss: 0.5658 - val_accuracy: 0.8421
Epoch 119/150
18/18 [==============================] - 0s 4ms/step - loss: 0.0553 - accuracy: 0.9855 - val_loss: 0.8346 - val_accuracy: 0.7778
Epoch 120/150
18/18 [==============================] - 0s 5ms/step - loss: 0.0630 - accuracy: 0.9798 - val_loss: 0.6098 - val_accuracy: 0.8070
Epoch 121/150
18/18 [==============================] - 0s 5ms/step - loss: 0.0562 - accuracy: 0.9855 - val_loss: 0.5981 - val_accuracy: 0.8187
Epoch 122/150
18/18 [==============================] - 0s 6ms/step - loss: 0.0465 - accuracy: 0.9827 - val_loss: 0.6102 - val_accuracy: 0.8129
Epoch 123/150

In [63]:
# evaluate the model
score = model.evaluate(x,y)
print((model.metrics_names[1]))

17/17 [==============================] - 0s 2ms/step - loss: 0.1967 - accuracy: 0.9400
accuracy


# Artificial Neural Network Model - Tuning of All Hyperparameters

# Tuning of Hyperparameter : Batch size, epochs, learning rate, Dropout, Activation, Kernel Initializer, Number of neurons in activation number(nueron1 and nueron2)

In [42]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import adam_v2
from keras.layers import Dropout

In [43]:
# Standardization
sc = StandardScaler()
sc.fit(x)
x_std = sc.transform(x)
pd.DataFrame(x_std).describe()1

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
count,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,...,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02
mean,-1.754024e-15,3.070830e-16,7.387171e-17,-3.865380e-17,2.005703e-16,3.362881e-16,-2.676776e-16,-2.841054e-16,-1.274502e-16,4.874674e-17,...,7.179943e-16,-1.933764e-16,-2.260174e-17,1.352883e-17,1.169277e-16,2.265542e-16,-2.596515e-16,1.443075e-16,6.253326e-16,4.024290e-16
std,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,...,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00
min,-1.304582e+01,-1.715608e+00,-2.179108e+00,-1.980578e+00,-2.876943e+00,-1.796637e+00,-2.021098e+00,-7.326831e-02,-2.020198e-01,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
25%,-8.063453e-02,-6.606652e-01,-4.448281e-01,-5.535954e-01,-5.842379e-01,-6.924563e-01,-7.361236e-01,-7.326831e-02,-2.020198e-01,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
50%,1.732292e-01,-4.020255e-02,4.691190e-01,-1.364774e-01,7.082076e-02,-1.403660e-01,-9.833712e-03,-7.326831e-02,-1.938429e-01,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,-7.060812e-01
75%,4.089598e-01,4.927389e-01,6.696628e-01,3.904086e-01,6.741643e-01,5.344111e-01,4.929823e-01,-7.326831e-02,-9.870852e-02,-4.435755e-01,...,-1.331035e-01,-2.006027e-01,-6.231770e-02,-2.568645e-01,-1.843909e-01,-3.415123e-01,-6.231770e-02,-4.402255e-02,-1.728597e-01,1.416268e+00
max,1.007353e+00,2.819865e+00,1.261610e+00,1.033538e+01,2.484195e+00,3.417549e+00,3.007063e+00,2.157228e+01,1.695111e+01,2.254407e+00,...,7.512952e+00,4.984977e+00,1.604681e+01,3.893103e+00,5.423261e+00,2.928152e+00,1.604681e+01,2.271563e+01,5.785038e+00,1.416268e+00


In [44]:
# Define the model
def create_model1(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 28, kernel_initializer = init,activation= activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2, input_dim = neuron1, activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation ='sigmoid'))
    
    adam = adam_v2.Adam(learning_rate = learning_rate)
    model.compile(loss ='binary_crossentropy', optimizer =  adam, metrics=['accuracy'])
    return model

In [64]:
# create the model
model1 = KerasClassifier(build_fn = create_model1, verbose = 0)

C:\Users\Sagar\AppData\Local\Temp/ipykernel_1744/944555759.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model1 = KerasClassifier(build_fn = create_model1, verbose = 0)


In [46]:
# Define the grid serch parameter
batch_size = [10,20]
epochs = [10,50]
learning_rate = [0.01,0.1]
dropout_rate = [0.1,0.2]
activation_function = ['softmax','tanh']
init = ['uniform','normal']
neuron1 = [4,8,]
neuron2 = [2,4,]

In [47]:
# make the dictionary of the grid search parameter
param_grids = dict(batch_size = batch_size,epochs = epochs,learning_rate = learning_rate,dropout_rate = dropout_rate,
                   activation_function = activation_function,init = init,neuron1 = neuron1,neuron2 = neuron2)

In [48]:
## Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model1, param_grid = param_grids, cv = KFold(), verbose = 10)
grid_result = grid.fit(x_std,y)

Fitting 5 folds for each of 256 candidates, totalling 1280 fits
[CV 1/5; 1/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.01, neuron1=4, neuron2=2
[CV 1/5; 1/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.01, neuron1=4, neuron2=2;, score=0.971 total time=   1.3s
[CV 2/5; 1/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.01, neuron1=4, neuron2=2
[CV 2/5; 1/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.01, neuron1=4, neuron2=2;, score=0.769 total time=   1.2s
[CV 3/5; 1/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.01, neuron1=4, neuron2=2
[CV 3/5; 1/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.0

[CV 1/5; 6/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.1, neuron1=4, neuron2=4;, score=0.933 total time=   1.2s
[CV 2/5; 6/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.1, neuron1=4, neuron2=4
[CV 2/5; 6/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.1, neuron1=4, neuron2=4;, score=0.673 total time=   1.2s
[CV 3/5; 6/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.1, neuron1=4, neuron2=4
[CV 3/5; 6/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.1, neuron1=4, neuron2=4;, score=0.524 total time=   1.2s
[CV 4/5; 6/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=uniform, learning_rate=0.1, neuron1=4, neuron2=4
[CV 4/5; 6/25

[CV 1/5; 11/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.01, neuron1=8, neuron2=2;, score=0.942 total time=   1.2s
[CV 2/5; 11/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.01, neuron1=8, neuron2=2
[CV 2/5; 11/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.01, neuron1=8, neuron2=2;, score=0.779 total time=   1.2s
[CV 3/5; 11/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.01, neuron1=8, neuron2=2
[CV 3/5; 11/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.01, neuron1=8, neuron2=2;, score=0.689 total time=   1.2s
[CV 4/5; 11/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.01, neuron1=8, neuron2=2
[CV 4/5

[CV 1/5; 16/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.1, neuron1=8, neuron2=4;, score=0.990 total time=   1.4s
[CV 2/5; 16/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.1, neuron1=8, neuron2=4
[CV 2/5; 16/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.1, neuron1=8, neuron2=4;, score=0.769 total time=   1.1s
[CV 3/5; 16/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.1, neuron1=8, neuron2=4
[CV 3/5; 16/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.1, neuron1=8, neuron2=4;, score=0.641 total time=   1.1s
[CV 4/5; 16/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=10, init=normal, learning_rate=0.1, neuron1=8, neuron2=4
[CV 4/5; 16/2

[CV 1/5; 21/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.1, neuron1=4, neuron2=2;, score=0.923 total time=   3.5s
[CV 2/5; 21/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.1, neuron1=4, neuron2=2
[CV 2/5; 21/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.1, neuron1=4, neuron2=2;, score=0.788 total time=   3.4s
[CV 3/5; 21/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.1, neuron1=4, neuron2=2
[CV 3/5; 21/256] END activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.1, neuron1=4, neuron2=2;, score=0.709 total time=   3.7s
[CV 4/5; 21/256] START activation_function=softmax, batch_size=10, dropout_rate=0.1, epochs=50, init=uniform, learning_rate=0.1, neuron1=4, neuron2=2
[CV 4/5

...

[CV 4/5; 233/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.01, neuron1=4, neuron2=2;, score=0.874 total time=   0.8s
[CV 5/5; 233/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.01, neuron1=4, neuron2=2
[CV 5/5; 233/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.01, neuron1=4, neuron2=2;, score=0.854 total time=   0.8s
[CV 1/5; 234/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.01, neuron1=4, neuron2=4
[CV 1/5; 234/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.01, neuron1=4, neuron2=4;, score=0.990 total time=   0.9s
[CV 2/5; 234/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.01, neuron1=4, neuron2=4
[CV 2/5; 234/256] E

[CV 5/5; 238/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.1, neuron1=4, neuron2=4;, score=0.845 total time=   0.9s
[CV 1/5; 239/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.1, neuron1=8, neuron2=2
[CV 1/5; 239/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.1, neuron1=8, neuron2=2;, score=0.904 total time=   0.8s
[CV 2/5; 239/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.1, neuron1=8, neuron2=2
[CV 2/5; 239/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.1, neuron1=8, neuron2=2;, score=0.750 total time=   0.9s
[CV 3/5; 239/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=10, init=normal, learning_rate=0.1, neuron1=8, neuron2=2
[CV 3/5; 239/256] END act

[CV 1/5; 244/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.01, neuron1=8, neuron2=4;, score=1.000 total time=   2.0s
[CV 2/5; 244/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.01, neuron1=8, neuron2=4
[CV 2/5; 244/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.01, neuron1=8, neuron2=4;, score=0.923 total time=   2.0s
[CV 3/5; 244/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.01, neuron1=8, neuron2=4
[CV 3/5; 244/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.01, neuron1=8, neuron2=4;, score=0.913 total time=   2.0s
[CV 4/5; 244/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=uniform, learning_rate=0.01, neuron1=8, neuron2=4
[CV 4/5; 244/

[CV 2/5; 249/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.01, neuron1=4, neuron2=2;, score=0.913 total time=   2.0s
[CV 3/5; 249/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.01, neuron1=4, neuron2=2
[CV 3/5; 249/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.01, neuron1=4, neuron2=2;, score=0.893 total time=   2.0s
[CV 4/5; 249/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.01, neuron1=4, neuron2=2
[CV 4/5; 249/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.01, neuron1=4, neuron2=2;, score=0.913 total time=   2.0s
[CV 5/5; 249/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.01, neuron1=4, neuron2=2
[CV 5/5; 249/256] E

[CV 3/5; 254/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.1, neuron1=4, neuron2=4;, score=0.845 total time=   2.0s
[CV 4/5; 254/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.1, neuron1=4, neuron2=4
[CV 4/5; 254/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.1, neuron1=4, neuron2=4;, score=0.825 total time=   2.0s
[CV 5/5; 254/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.1, neuron1=4, neuron2=4
[CV 5/5; 254/256] END activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.1, neuron1=4, neuron2=4;, score=0.893 total time=   2.0s
[CV 1/5; 255/256] START activation_function=tanh, batch_size=20, dropout_rate=0.2, epochs=50, init=normal, learning_rate=0.1, neuron1=8, neuron2=2
[CV 1/5; 255/256] END act

In [49]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.9496265888214112, using {'activation_function': 'tanh', 'batch_size': 20, 'dropout_rate': 0.2, 'epochs': 50, 'init': 'normal', 'learning_rate': 0.01, 'neuron1': 4, 'neuron2': 4}
0.8199215888977051,0.09588069638068977 with: {'activation_function': 'softmax', 'batch_size': 10, 'dropout_rate': 0.1, 'epochs': 10, 'init': 'uniform', 'learning_rate': 0.01, 'neuron1': 4, 'neuron2': 2}
0.8353995561599732,0.09697194771058361 with: {'activation_function': 'softmax', 'batch_size': 10, 'dropout_rate': 0.1, 'epochs': 10, 'init': 'uniform', 'learning_rate': 0.01, 'neuron1': 4, 'neuron2': 4}
0.8334578037261963,0.09733797934370714 with: {'activation_function': 'softmax', 'batch_size': 10, 'dropout_rate': 0.1, 'epochs': 10, 'init': 'uniform', 'learning_rate': 0.01, 'neuron1': 8, 'neuron2': 2}
0.821863317489624,0.08577931062943986 with: {'activation_function': 'softmax', 'batch_size': 10, 'dropout_rate': 0.1, 'epochs': 10, 'init': 'uniform', 'learning_rate': 0.01, 'neuron1': 8, 'neuron2': 4}
0.

# Final model

In [50]:
# create model
final_model = Sequential()
final_model.add(Dense(8, input_dim=28, kernel_initializer ='normal', activation='tanh'))
final_model.add(Dropout(0.2))
final_model.add(Dense(4, kernel_initializer='uniform', activation='softmax'))
final_model.add(Dropout(0.2))
final_model.add(Dense(1, kernel_initializer='uniform', activation='linear'))

adam=adam_v2.Adam(learning_rate = 0.01)
## compile model
final_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

## fit the model 
final_model.fit(x_std,y, epochs=50, batch_size=10)

Epoch 1/50
52/52 [==============================] - 1s 2ms/step - loss: 0.6700 - mse: 0.6700
Epoch 2/50
52/52 [==============================] - 0s 2ms/step - loss: 0.5470 - mse: 0.5470
Epoch 3/50
52/52 [==============================] - 0s 2ms/step - loss: 0.4474 - mse: 0.4474
Epoch 4/50
52/52 [==============================] - 0s 2ms/step - loss: 0.3767 - mse: 0.3767
Epoch 5/50
52/52 [==============================] - 0s 2ms/step - loss: 0.3157 - mse: 0.3157
Epoch 6/50
52/52 [==============================] - 0s 2ms/step - loss: 0.2725 - mse: 0.2725
Epoch 7/50
52/52 [==============================] - 0s 2ms/step - loss: 0.2485 - mse: 0.2485
Epoch 8/50
52/52 [==============================] - 0s 2ms/step - loss: 0.2347 - mse: 0.2347
Epoch 9/50
52/52 [==============================] - 0s 2ms/step - loss: 0.2220 - mse: 0.2220
Epoch 10/50
52/52 [==============================] - 0s 2ms/step - loss: 0.2138 - mse: 0.2138
Epoch 11/50
52/52 [==============================] - 0s 2ms/step - lo

In [51]:
# evaluate the model
score = final_model.evaluate(x,y)
print((final_model.metrics_names[1]))

17/17 [==============================] - 0s 2ms/step - loss: 0.4998 - mse: 0.4998
mse
